In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import time
from datetime import datetime
import scipy.spatial
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
# We're going to be calculating memory usage a lot,
# so we'll create a function to save us some time!

def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [3]:
dtypes = pd.Series({'idaviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))
postulaciones = pd.read_csv('Data/Postulaciones/Postulaciones-merge.csv',
                            dtype=column_types,parse_dates=['fechapostulacion'],infer_datetime_format=True)
postulaciones = postulaciones.drop(columns=postulaciones.columns[0])

In [4]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603752 entries, 0 to 6603751
Data columns (total 3 columns):
idaviso             uint32
idpostulante        object
fechapostulacion    datetime64[ns]
dtypes: datetime64[ns](1), object(1), uint32(1)
memory usage: 126.0+ MB


In [5]:
dtypes = pd.Series({'idpostulante': np.dtype('object'), 'sexo': CategoricalDtype(categories=['FEM', 'MASC', 'NO_DECLARA'], ordered=False)})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_genero_edad = pd.read_csv('Data/Postulantes-Genero-Edad/postulantes_genero_y_edad-merge.csv',
                                    dtype=column_types,parse_dates=['fechanacimiento'],
                                      infer_datetime_format=True)
postulantes_genero_edad = postulantes_genero_edad.drop(columns=['Unnamed: 0'])

In [6]:
dtypes = pd.Series({'estado': CategoricalDtype(categories=['Abandonado', 'En Curso', 'Graduado'], ordered=False), 'idpostulante': np.dtype('object'),
                    'nombre': CategoricalDtype(categories=['Doctorado', 'Master', 'Otro', 'Posgrado', 'Secundario',
                  'Terciario/Técnico', 'Universitario'],
                 ordered=False)})


dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_educacion = pd.read_csv('Data/Postulantes-Educacion/postulantes_educacion-merge.csv',
                                    dtype=column_types)
postulantes_educacion = postulantes_educacion.drop(columns=['Unnamed: 0','nombre_numerico'])

In [7]:
#Hago un merge de educacion y genero-edad
postulantes_merge = pd.merge(postulantes_genero_edad,postulantes_educacion,how='left',on='idpostulante')

In [8]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 5 columns):
idpostulante       504407 non-null object
fechanacimiento    504407 non-null object
sexo               504407 non-null category
nombre             447909 non-null category
estado             447909 non-null category
dtypes: category(3), object(2)
memory usage: 13.0+ MB


In [9]:
postulantes_merge.head()

,idpostulante,fechanacimiento,sexo,nombre,estado
0,6MM,1985-01-01,MASC,NaN,NaN
1,Nzz,nan,NO_DECLARA,NaN,NaN
2,ZX1,nan,NO_DECLARA,NaN,NaN
3,Nq5,nan,NO_DECLARA,NaN,NaN
4,ebE,1952-07-07,MASC,Posgrado,Graduado


In [10]:
#Voy a agregar una columna con la edad ya calculada(estimando por 2017-año nacimiento) para cada registro
postulantes_merge["año"] = postulantes_merge[postulantes_merge["fechanacimiento"] != 'nan']\
        ["fechanacimiento"].apply(lambda x: x[0:4])
postulantes_merge["edad"] = postulantes_merge[postulantes_merge["año"].notna()] \
        ["año"].apply(lambda x: 2017 - int(x))

In [11]:
#Elimino columna fechanacimiento y año que ya no me sirven
postulantes_merge = postulantes_merge.drop(columns=['fechanacimiento','año'])

In [12]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 5 columns):
idpostulante    504407 non-null object
sexo            504407 non-null category
nombre          447909 non-null category
estado          447909 non-null category
edad            477918 non-null float64
dtypes: category(3), float64(1), object(1)
memory usage: 13.0+ MB


In [13]:
#Me quedo con los registros con todos los datos
postulantes_merge = postulantes_merge[postulantes_merge['nombre'].notnull() & 
                                      postulantes_merge['estado'].notnull() & 
                                     postulantes_merge['edad'].notnull()]
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438357 entries, 4 to 504405
Data columns (total 5 columns):
idpostulante    438357 non-null object
sexo            438357 non-null category
nombre          438357 non-null category
estado          438357 non-null category
edad            438357 non-null float64
dtypes: category(3), float64(1), object(1)
memory usage: 11.3+ MB


In [14]:
#Vamos a codificar cada columna que sea de strings
lb_make1 = LabelEncoder()
postulantes_merge["estado_code"] = lb_make1.fit_transform(postulantes_merge["estado"])

lb_make2 = LabelEncoder()
postulantes_merge["sexo_code"] = lb_make2.fit_transform(postulantes_merge["sexo"])

lb_make3 = LabelEncoder()
postulantes_merge["nombre_code"] = lb_make3.fit_transform(postulantes_merge["nombre"])


In [15]:
#Dropeo las columnas que use para codificar
postulantes_merge = postulantes_merge.drop(columns=['sexo','nombre','estado'])
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438357 entries, 4 to 504405
Data columns (total 5 columns):
idpostulante    438357 non-null object
edad            438357 non-null float64
estado_code     438357 non-null int64
sexo_code       438357 non-null int64
nombre_code     438357 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 20.1+ MB


In [16]:
postulantes_merge_int = postulantes_merge.select_dtypes(include=['int','float'])
converted_int = postulantes_merge_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(postulantes_merge_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([postulantes_merge_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

16.72 MB
5.43 MB


,before,after
uint8,NaN,3.0
uint16,NaN,1.0
int64,3.0,NaN
float64,1.0,NaN


In [17]:
optimized_postulantes_merge = postulantes_merge.copy()

optimized_postulantes_merge[converted_int.columns] = converted_int

print(mem_usage(postulantes_merge))
print(mem_usage(optimized_postulantes_merge))

43.39 MB
32.10 MB


In [18]:
optimized_postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438357 entries, 4 to 504405
Data columns (total 5 columns):
idpostulante    438357 non-null object
edad            438357 non-null uint16
estado_code     438357 non-null uint8
sexo_code       438357 non-null uint8
nombre_code     438357 non-null uint8
dtypes: object(1), uint16(1), uint8(3)
memory usage: 8.8+ MB


In [19]:
optimized_postulantes_merge.to_csv('Data/optimized_postulantes_merge.csv')

In [20]:
intereses_vistas = pd.read_csv('Data/unique_postulantes_con_intereses_vistas.csv')

In [22]:
intereses_vistas.head()

,idpostulante,nombre_zona_interes,nivel_laboral_interes,tipo_de_trabajo_interes,nombre_area_interes
0,YjVJQ6Z,-7,-4,-1,-175
1,BmVpYoR,NaN,NaN,NaN,NaN
2,wVkBzZd,-7,-2,-1,-120
3,OqmP9pv,-7,-4-2,-1,-175
4,DrpbXDP,NaN,NaN,NaN,NaN
